<a href="https://colab.research.google.com/github/yefang008514/machinelearning/blob/master/%E2%80%9C%E6%9C%A8%E8%96%AF%E5%8F%B6%E7%96%BE%E7%97%85%E8%AF%86%E5%88%AB_1012_draft_ipynb%E2%80%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os #连接到我的云盘
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)

In [ ]:
import os, sys, glob, argparse
import pandas as pd
import numpy as np
from tqdm import tqdm

import time, datetime
import pdb, traceback

import cv2
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn          #
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

In [ ]:
# import shutil
# # os.mkdir('/content/sample_data/mushuye')
# # os.mkdir('/content/sample_data/mushuye/train')
# shutil.copytree('/content/drive/MyDrive/mushuye/train','/content/sample_data/mushuye/train')

In [ ]:
# os.listdir('/content/sample_data/mushuye/train')

[]

In [ ]:
# import shutil
# os.mkdir('/content/mushuye/data/train/')
# os.mkdir('/content/mushuye/data/train/CBB')
# os.mkdir('/content/mushuye/data/train/CBSD')
# os.mkdir('/content/mushuye/data/train/CGM')
# os.mkdir('/content/mushuye/data/train/CMD')
# os.mkdir('/content/mushuye/data/train/H')
# os.mkdir('/content/mushuye/data/train/AL')
# img_name=[]
# for i in range(0,6):
#   img_name.append(train_table[train_table['label']==i]['image_id'])

# for j in range(0,6):#读取图片路径并保存
#   for k in img_name[j]:
#     path = '/content/mushuye/data/train_img/'+k
#     if j==0: 
#       path_to = '/content/mushuye/data/train/CBB/'+k
#     elif j==1:
#       path_to = '/content/mushuye/data/train/CBSD/'+k
#     elif j==2:
#       path_to = '/content/mushuye/data/train/CGM/'+k
#     elif j==3:
#       path_to = '/content/mushuye/data/train/CMD/'+k
#     elif j==4:
#       path_to = '/content/mushuye/data/train/H/'+k
#     elif j==5:
#       path_to = '/content/mushuye/data/train/AL/'+k
#     # print(path,j,path_to) 
#     shutil.copy(path,path_to)

In [ ]:
train_jpg = glob.glob(r'/content/drive/MyDrive/mushuye/train/*/*')
np.random.shuffle(train_jpg)
train_jpg = np.array(train_jpg)
    
class QRDataset(Dataset):
    def __init__(self, img_path, transform=None):
        self.img_path = img_path
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    
    def __getitem__(self, index):
        start_time = time.time()
        img = Image.open(self.img_path[index]).convert('RGB')
        #图片给标签
        lbl_dict = {
             'CBB': 0,
             'CBSD': 1,
             'CGM': 2,
             'CMD': 3,
             'H': 4,
             'AL': 5,
             }
        if self.transform is not None:
            img = self.transform(img)
        
        if 'test' in self.img_path[index]:
            return img, torch.from_numpy(np.array(0))
        else:
            lbl_int = lbl_dict[self.img_path[index].split('/')[-2]]
            return img, torch.from_numpy(np.array(lbl_int))
    
    def __len__(self):
        return len(self.img_path)

In [ ]:
train_jpg

array(['/content/drive/MyDrive/mushuye/train/CBB/1546870332.jpg',
       '/content/drive/MyDrive/mushuye/train/AL/Train_1437.jpg',
       '/content/drive/MyDrive/mushuye/train/H/2416129952.jpg', ...,
       '/content/drive/MyDrive/mushuye/train/CBB/3142477676.jpg',
       '/content/drive/MyDrive/mushuye/train/AL/Train_121.jpg',
       '/content/drive/MyDrive/mushuye/train/H/2224228735.jpg'],
      dtype='<U56')

In [ ]:
class XunFeiNet(nn.Module):
    def __init__(self):
        super(XunFeiNet, self).__init__()
                
        model = models.resnet18(True)        #https://pytorch.org/vision/stable/models.html RESNET
        model.avgpool = nn.AdaptiveAvgPool2d(1)   #POOLING
        model.fc = nn.Linear(512, 6)        #FULLY CONNETCTED NETWORK https://pytorch.org/docs/stable/nn.html#linear-layers
        self.resnet = model
         
    def forward(self, img):        
        out = self.resnet(img)
        return out

def validate(val_loader, model, criterion):
    model.eval()
    acc1 = []
    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(val_loader):
            input = input.cuda()
            target = target.cuda()
            
            output = model(input)
            loss = criterion(output, target)
            acc1.append((output.argmax(1) == target).float().mean().item())

        print(' * Val Acc@1 {0}'.format(np.mean(acc1)))
        return np.mean(acc1)

def predict(test_loader, model, tta=10):
    model.eval()
    
    test_pred_tta = None
    for _ in range(tta):
        test_pred = []
        with torch.no_grad():
            end = time.time()
            for i, (input, target) in enumerate(test_loader):
                input = input.cuda()
                target = target.cuda()

                output = model(input)
                output = output.data.cpu().numpy()

                test_pred.append(output)
        test_pred = np.vstack(test_pred)
    
        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred
    
    return test_pred_tta

def train(train_loader, model, criterion, optimizer, epoch):
    model.train()

    end = time.time()
    acc1 = []
    for i, (input, target) in enumerate(train_loader):
        input = input.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)
        output = model(input)
        loss = criterion(output, target)

        acc1.append((output.argmax(1) == target).float().mean().item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 100 == 0:
            print('Train: {0}'.format(np.mean(acc1)))

In [ ]:
skf = KFold(n_splits=5, random_state=2334, shuffle=True)
for flod_idx, (train_idx, val_idx) in enumerate(skf.split(train_jpg, train_jpg)):
    
    train_loader = torch.utils.data.DataLoader(
        QRDataset(train_jpg[train_idx][:],
                transforms.Compose([
                            transforms.Resize((124, 124)),        
                            transforms.RandomAffine(10),
                            transforms.ColorJitter(hue=.05, saturation=.05),
                            transforms.RandomHorizontalFlip(),
                            transforms.RandomVerticalFlip(),
                            # transforms.Resize((124, 124)),
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                            # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
            ])
        ), batch_size=50, shuffle=True, num_workers=5, pin_memory=True
    )
    
    val_loader = torch.utils.data.DataLoader(
        QRDataset(train_jpg[val_idx][:1000],
                transforms.Compose([
                            # transforms.Resize((300, 400)),
                            transforms.Resize((124, 124)),
                            # transforms.RandomCrop((88, 88)),
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                            # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
            ])
        ), batch_size=10, shuffle=False, num_workers=5, pin_memory=True
    )
        
    model = XunFeiNet().cuda()
    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = torch.optim.SGD(model.parameters(), 0.003)
    best_acc = 0.0
    for epoch in range(10):
        print('\nEpoch: ', epoch)

        train(train_loader, model, criterion, optimizer, epoch)
        val_acc = validate(val_loader, model, criterion)
        
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), './resnet18_fold{0}.pt'.format(flod_idx))
            
    break

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]


Epoch:  0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train: 0.03999999910593033


KeyboardInterrupt: ignored

In [ ]:
#保存模型
torch.save(model.state_dict(), '/content/drive/MyDrive/resnet18_fold{0}.pt'.format(flod_idx))

# 下面没有用

In [ ]:
test_jpg = glob.glob('./Datawhale_人脸情绪识别_数据集/test/*')
test_jpg = np.array(test_jpg)
test_jpg.sort()

test_loader = torch.utils.data.DataLoader(
        QRDataset(test_jpg,
                transforms.Compose([
                            transforms.RandomHorizontalFlip(),
                            transforms.RandomVerticalFlip(),
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        ), batch_size=50, shuffle=False, num_workers=5, pin_memory=True
)
        
model = XunFeiNet().cuda()
model.load_state_dict(torch.load('resnet18_fold0.pt'))
test_pred = predict(test_loader, model, 5)


# test_csv = pd.DataFrame()
# test_csv['ID'] = list(range(0, 3082))
# test_csv['Label'] = np.argmax(test_pred, 1)
# test_csv['Label'] = test_csv['Label'].map({1:'pos', 0:'neg'})
# test_csv.to_csv('tmp.csv', index=None)

In [ ]:
cls_name = np.array(['angry', 'disgusted', 'fearful', 'happy','neutral', 'sad', 'surprised'])
submit_df = pd.DataFrame({'name': test_jpg, 'label': cls_name[test_pred.argmax(1)]})
submit_df['name'] = submit_df['name'].apply(lambda x: x.split('/')[-1])

In [ ]:
submit_df = submit_df.sort_values(by='name')
submit_df.to_csv('pytorch_submit.csv', index=None)